# Kernel PCA on Euclidean dataset

In [1]:
#import 
import os, sys
sys.path.insert(1, os.path.abspath(".."))
from sklearn.manifold import TSNE
import pickle
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA, KernelPCA
from copy import deepcopy
from mpl_toolkits.mplot3d import axes3d
import pandas as pd
from matplotlib.path import Path
import matplotlib.patches as patches
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [2]:
EXP_NAME = "experiment0"  

In [3]:
path = "../data_generation/generated_data/" + EXP_NAME
with open(path + "/maze.pkl", 'rb') as file:
    maze = pickle.load(file) #Maze is common to both
with open(path + "/trajectory.pkl", 'rb') as file:
    traj = pickle.load(file)
with open(path + "/placeCells.pkl", 'rb') as file:
    placeCells = pickle.load(file)

In [4]:
 #build useful idx arrays
trajectory = np.array([traj.x_traj, traj.y_traj])
long_traj_maze_config = np.empty(trajectory.shape[1])
long_traj_maze_config[0:traj.traj_cut_idx[0]] = traj.corr_maze_config[0]
for i in range(len(traj.corr_maze_config)):
    cur_conf = traj.corr_maze_config[i]
    long_traj_maze_config[traj.traj_cut_idx[i]:traj.traj_cut_idx[i + 1]] = cur_conf
long_edge_pos = np.empty(trajectory.shape[1])
long_edge_pos[0:traj.traj_cut_idx[0]] = traj.corr_maze_config[0]
for i in range(len(traj.edge_position)):
    # nb_goals = (len(maze.goals[cur_conf]))
    long_edge_pos[traj.traj_cut_idx[i]:traj.traj_cut_idx[i + 1]] = traj.edge_position[i]

In [5]:
FR = placeCells.firingRates
edge = long_edge_pos
traj_2d= np.vstack((traj.x_traj, traj.y_traj)).T
maze_config = long_traj_maze_config


In [6]:
with open('../rel_graph_distance/dge_', 'rb') as file:
    dg = pickle.load(file)

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
#useful trajectory functions
def extract_kpca_space_trajectory(traj, goal, maze_config, pc, kpca):
    idx = np.where((np.array(traj.edge_position) == goal) & (np.array(traj.corr_maze_config) == maze_config))[0][0]
    neural_activity = pc.firingRates[traj.traj_cut_idx[idx]:traj.traj_cut_idx[idx+1], :]
    return kpca.transform(neural_activity)   

def extract_maze_trajectory(traj, goal, maze_config):
    idx = np.where((np.array(traj.edge_position) == goal) & (np.array(traj.corr_maze_config) == maze_config))[0][1]
    traj_x = traj.x_traj[traj.traj_cut_idx[idx]:traj.traj_cut_idx[idx+1]]
    traj_y = traj.y_traj[traj.traj_cut_idx[idx]:traj.traj_cut_idx[idx+1]]
    
    return traj_x, traj_y

#useful plotting functions
def ax_plot_traj(ax, traj_x_array, traj_y_array, maze, config, traj_col, maze_color="Greens", s_color="y"):
    #ax.title(title)
    col = sns.color_palette("colorblind")  
    ax.plot(traj_x_array, traj_y_array, color = traj_col)
    ax.plot(traj_x_array[0], traj_y_array[0], 'o', color = s_color)
    ax.plot(traj_x_array[-1], traj_y_array[-1], 'v', color = s_color)
    ax.set_xlim([0, maze.N])
    ax.set_ylim([0, maze.N])
    ax.grid()
    
        
        
def ax_plot_maze(ax, maze, config, plot_goals=[True, True, True], plot_home=True, maze_col = "Greys", ls = "-", hatch = None):
    col = sns.color_palette("colorblind")  
    patch = patches.PathPatch(maze.trialOctoMaze[config], facecolor='None', lw=3, linestyle = ls, hatch=hatch)
    ax.add_patch(patch)
    ax.imshow(maze.trialMazeFlags[config, ::-1], cmap=maze_col, extent=[0, maze.N, 0, maze.N], alpha=0.1)
    h = np.array(maze.nodeList[config]["A"]) + 0.5
    if plot_home:
        ax.scatter(h[0], h[1], s = 500, alpha = 1, marker = "P", color = col[0])
    for k in range(3): #goals
        gg = ["C", "D", "E"]
        if plot_goals[k]:
            g = np.array(maze.nodeList[config][gg[k]])+0.5
            ax.scatter(g[0], g[1], s = 500, color = col[k+2], marker = "8", alpha= 0.5) 

# 2D Kernel PCA

In [ ]:
SAMPLE_SIZE=20000
idx = np.random.choice(len(FR), SAMPLE_SIZE)
Xe_small = FR[idx]

kpca = KernelPCA(n_components=2)
z = kpca.fit_transform(Xe_small)

In [ ]:
col = np.zeros((3, Xe_small.shape[0]))
col[0, :] = traj_2d[idx, 0] #traj_train[:, 0]
col[2, :] = traj_2d[idx, 1] #traj_train[:, 1]
col = col/7

In [ ]:
s_i = np.arange(len(idx))
np.random.shuffle(s_i)

plt.scatter(z[s_i, 0], z[s_i, 1], c=col[:, s_i].T)

In [ ]:
plt.scatter(z[s_i, 0], z[s_i, 1], c=dg[s_i])

In [ ]:
plt.scatter(z[:, 0], z[:, 1], c=edge[idx])

# Trajectories

In [ ]:
maze_config = maze_config[:SAMPLE_SIZE]
edge = edge[:SAMPLE_SIZE]
import seaborn as sns
col = sns.color_palette("colorblind")
col

In [ ]:
n_maze=3

#Plot trajectories in latent space
#EUCL
plt.figure()
fig, axs = plt.subplots(2, n_maze, figsize = (15, 10))

for m in range(n_maze):
    im = axs[0, m].scatter(z[:, 0], z[:, 1], color="grey")


for k in range(n_maze): #maze config
        #idx = np.where(maze_config_e == k)
    home = maze.nodeList[k]["A"]
    idx = np.where(maze_config == k)
    patch = patches.PathPatch(maze.trialOctoMaze[k], facecolor='None', lw=3)
    axs[1, k].add_patch(patch)
    
  
    
    for g in range(3): #goal edge
        traj_act_latent = extract_kpca_space_trajectory(traj, goal=g, maze_config=k, pc=placeCells, kpca=kpca)
        
        #traj_act_latent= model.transform(traj)#traj_act_latent.detach().numpy())

        axs[0, k].scatter(traj_act_latent[:, 0], traj_act_latent[:, 1], color=col[g+2], label = "maze %s, goal %s"%(k, g))
        axs[0, k].plot(traj_act_latent[0, 0], traj_act_latent[0, 1], 'yo', markersize = 8)
        axs[0, k].plot(traj_act_latent[-1, 0], traj_act_latent[-1, 1], 'yx',  markersize = 8)        
        
        
        traj_x_array, traj_y_array = extract_maze_trajectory(traj, g, k)
        ax_plot_traj(axs[1, k], traj_x_array, traj_y_array, maze, k, traj_col = col[g+2])

In [ ]:
plt.figure()
fig, axs = plt.subplots(2, 3, figsize = (15, 10))

for g in range(3): #goal edge
    idx_g = np.where(edge == g)
    for m in range(3):        
        im = axs[0, m].scatter(z[idx_g, 0], z[idx_g, 1], alpha = 1, color = lighten_color(col[g+2]))

    for k in range(10): #maze config
        traj_act_latent = extract_kpca_space_trajectory(traj, goal=g, maze_config=k, pc=placeCells, kpca= kpca)

        axs[0, g].plot(traj_act_latent[:, 0], traj_act_latent[:, 1], label = "maze %s, goal %s"%(k, g), c=col[g+2])
        axs[0, g].plot(traj_act_latent[0, 0], traj_act_latent[0, 1], 'yo', markersize = 8)
        axs[0, g].plot(traj_act_latent[-1, 0], traj_act_latent[-1, 1], 'yx',  markersize = 8)

        traj_x_array, traj_y_array = extract_maze_trajectory(traj, g, k)
        ax_plot_traj(axs[1, g], traj_x_array, traj_y_array, maze, k, traj_col = col[g+2])#, maze_color = colors[k])

        #patch = patches.PathPatch(maze.trialOctoMaze[k], facecolor='None', lw=3)
        #axs[1, g].add_patch(patch)
        ax_plot_maze(axs[1, g], maze, k)

# 3D kernel PCA

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [ ]:
SAMPLE_SIZE=20000
idx = np.random.choice(len(FR), SAMPLE_SIZE)
Xe_small = FR[idx]

kpca = KernelPCA(n_components=3, kernel="rbf")
z = kpca.fit_transform(Xe_small)

In [ ]:
df = pd.DataFrame(z)
df = df.rename(columns={'0': 'dim1', '1': 'dim2', "2":"dim3"})

col = np.zeros((3, df.index.shape[0]))
col[0, :] =  traj.x_traj[df.index]
col[2, :] =  traj.y_traj[df.index]
col = col/7*256


marker_dict = dict(size=3,
        color= [f'rgb({col[0, i]}, {col[1, i]}, {col[2, i]})' for i in range(col.shape[1])],  # set color to an array/list of desired values
        opacity=0.8)

fig = go.Figure()
fig.add_scatter3d(x=df[0], y=df[1], z=df[2],mode='markers', marker=marker_dict) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[df[0].min(),df[0].max()],),
                     yaxis = dict(nticks=4, range=[df[1].min(),df[1].max()],),
                     zaxis = dict(nticks=4, range=[df[2].min(),df[2].max()],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10), 
    title={
        'text': "Eucl",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()


#legend plot
cp1 = np.linspace(0,1)
cp2 = np.linspace(0,1)
Cp1, Cp2 = np.meshgrid(cp1,cp2)
C0 = np.zeros_like(Cp1)# make RGB image, p1 to red channel, p2 to blue channel
Legend = np.dstack((Cp1, C0, Cp2))

# parameters range between 0 and 7
plt.figure(figsize=[1, 1])
plt.imshow(Legend, origin="lower", extent=[0,7,0,7])
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Euclidean Coordinates", fontsize=10)

In [ ]:
df = pd.DataFrame(z)
df = df.rename(columns={'0': 'dim1', '1': 'dim2', "2":"dim3"})



marker_dict = dict(size=3,
        color= dg[df.index],  # set color to an array/list of desired values
        opacity=0.8)

fig = go.Figure()
fig.add_scatter3d(x=df[0], y=df[1], z=df[2],mode='markers', marker=marker_dict) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[df[0].min(),df[0].max()],),
                     yaxis = dict(nticks=4, range=[df[1].min(),df[1].max()],),
                     zaxis = dict(nticks=4, range=[df[2].min(),df[2].max()],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10), 
    title={
        'text': "Eucl",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

In [ ]:
df = pd.DataFrame(z)
df = df.rename(columns={'0': 'dim1', '1': 'dim2', "2":"dim3"})



marker_dict = dict(size=3,
        color= edge[df.index],  # set color to an array/list of desired values
        opacity=0.8)

fig = go.Figure()
fig.add_scatter3d(x=df[0], y=df[1], z=df[2],mode='markers', marker=marker_dict) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[df[0].min(),df[0].max()],),
                     yaxis = dict(nticks=4, range=[df[1].min(),df[1].max()],),
                     zaxis = dict(nticks=4, range=[df[2].min(),df[2].max()],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10), 
    title={
        'text': "Eucl",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()